In [1]:
import pandas as pd
import os
from pathlib import Path
#from multiprocessing import Pool, Lock
#import pydicom
import glob
import random
import string
import numpy as np
import pickle
import time
import sys
import hashlib

def hash_value(value, hash_key):
    return hashlib.sha256((str(value) + hash_key).encode()).hexdigest()

In [3]:
year = str(2014)
path_to_mastertable = Path('/labs/kamaleswaranlab/MODS/Data/Emory_Data/RKENCOUNTERS_BEDLOCATION.csv')
all_meta_columns = ['AccessionNumber', 'ContentDate', 'ContentDescription', 'ContentLabel', 'ContentTime',\
                    'ImageHorizontalFlip', 'ImageRotation', 'InstanceNumber', 'IssuerOfPatientID', 'Manufacturer', \
                    'ManufacturerModelName', 'Modality', 'PatientID', 'PatientSex', 'PresentationCreationDate', \
                        'PresentationCreationTime', 'ReferringPhysicianName', 'RescaleIntercept', 'RescaleSlope',\
                        'SOPClassUID', 'SOPInstanceUID', 'SeriesDate', 'SeriesDescription', 'SeriesInstanceUID', \
                        'SeriesNumber', 'SeriesTime', 'SoftwareVersions', 'SpecificCharacterSet', 'StationName', \
                        'StudyDate', 'StudyInstanceUID', 'StudyTime', 'file', 'has_pix_array', 'category', \
                        'AcquisitionDate', 'AcquisitionDateTime', 'AcquisitionTime', 'BitsAllocated',\
                        'BitsStored', 'Columns', 'HighBit', 'ImageType', 'InstitutionName', 'LossyImageCompression',\
                    'PatientOrientation', 'PhotometricInterpretation', 'PixelRepresentation', 'ProcessingFunction',\
                    'Rows', 'SamplesPerPixel', 'StudyDescription', 'StudyID', 'AcquisitionNumber', 'ConvolutionKernel',\
                'DataCollectionDiameter', 'DeviceSerialNumber', 'DistanceSourceToDetector', 'DistanceSourceToPatient', \
                    'Exposure', 'ExposureTime', 'ImageOrientationPatient', 'ImagePositionPatient', 'InstanceCreationDate',\
                'InstanceCreationTime', 'RescaleType', 'RevolutionTime', 'RotationDirection', 'ScanOptions', 'SliceLocation', \
                    'SliceThickness', 'SpacingBetweenSlices', 'EMPI']

columns_to_deid = ['AccessionNumber',  'InstanceNumber', 'IssuerOfPatientID', \
                    'PatientID','SOPClassUID', 'SOPInstanceUID', 'StudyInstanceUID',  'InstitutionName',  'EMPI']

all_mastertable_columns = ['PAT_ID', 'EMPI_NBR', 'ENCOUNTER_NBR', 'ENCOUNTER_ID', 'BED_LOCATION_START', 'BED_LOCATION_END', 'BED_UNIT', 'BED_ROOM', 'BED_ID', 'BED_LABEL', 'HOSPITAL_SERVICE', 'ACCOMODATION_CODE', 'ACCOMODATION_DESCRIPTION']
columns_to_read = ['PAT_ID', 'EMPI_NBR', 'ENCOUNTER_NBR', 'ENCOUNTER_ID', 'BED_LOCATION_START', 'BED_LOCATION_END']

mastertable = pd.read_csv(path_to_mastertable, usecols=  columns_to_read)

path_to_combined_meta = Path('/labs/collab/Imaging/Imaging-PHI/Emory_Images/Meta/Combined_Meta_' + year + '.pkl')
combined_meta = pd.read_pickle(path_to_combined_meta)

path_to_combined_radiology_notes = Path('/labs/collab/Imaging/Imaging-PHI/Emory_Images/Meta/Combined_Radiology_Notes.csv')
radiology_notes = pd.read_csv(path_to_combined_radiology_notes)



/tmp/ipykernel_2122492/534018347.py:25: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  mastertable = pd.read_csv(path_to_mastertable, usecols=  columns_to_read)
/tmp/ipykernel_2122492/534018347.py:31: DtypeWarning: Columns (0,1,2,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  radiology_notes = pd.read_csv(path_to_combined_radiology_notes)


In [16]:
radiology_notes.shape

(5099251, 11)

In [8]:
radiology_notes.head()

,ACC_NBR,PATIENT_ID,EMPI_NBR,ENCNTR_ID,HNAM_DOCUMENT_CLINICAL_ID,HNAM_DOCUMENT_CLINICAL_NM,DAY_VERIFIED,DOC_TEXT,DOC_ABSTRACT,EVENT_DOCUMENT_DESC,EVENT_DOCUMENT_KEY
0,00006DX190104831,18007243,803350,156619493.0,2228789^EVENT,XR Chest 1 View Portable,06/28/2019,REPORT\r\rEXAM: XR Chest 1 View Portable\n\nES...,REPORT\r\rEXAM: XR Chest 1 View Portable\n\nES...,00006DX20190104831,127041997.0
1,00002DX200136398,27595956,5889242,174163224.0,2228795^EVENT,XR Chest 2 Views PA + Lat Stnd Protocol,07/23/2020,REPORT\r\rEXAM: XR Chest 2 Views PA + Lat Stnd...,REPORT\r\rEXAM: XR Chest 2 Views PA + Lat Stnd...,00002DX20200136398,140584055.0
2,00001CT190035194,18095426,891532,149573470.0,1460399309^EVENT,CT Neuro Spine Outside Image Ref Only,02/02/2019,CT NEURO SPINE OUTSIDE IMAGE REF ONLY\r\rRefer...,CT NEURO SPINE OUTSIDE IMAGE REF ONLY\r\rRefer...,00001CT20190035194,115489211.0
3,00002DX200041549,18302477,1098583,167980557.0,2228795^EVENT,XR Chest 2 Views PA + Lat Stnd Protocol,02/25/2020,ADDEND\r\rThis is the incorrect report as this...,ADDEND\r\rThis is the incorrect report as this...,00002DX20200041549,135818096.0
4,00002DX200138873,19065136,1861242,174284902.0,2228795^EVENT,XR Chest 2 Views PA + Lat Stnd Protocol,07/27/2020,REPORT\r\rEXAM: XR Chest 2 Views PA + Lat Stnd...,REPORT\r\rEXAM: XR Chest 2 Views PA + Lat Stnd...,00002DX20200138873,140714001.0


In [20]:
combined_meta

,AccessionNumber,ContentDate,ContentDescription,ContentLabel,ContentTime,ImageHorizontalFlip,ImageRotation,InstanceNumber,IssuerOfPatientID,Manufacturer,...,InstanceCreationDate,InstanceCreationTime,RescaleType,RevolutionTime,RotationDirection,ScanOptions,SliceLocation,SliceThickness,SpacingBetweenSlices,EMPI
0,00001MG160004625,20160107.0,None,None,74106.0,None,NaN,12.0,SITE_DEFAULT_LOCAL,"HOLOGIC, Inc.",...,NaN,NaN,US,NaN,None,None,NaN,NaN,NaN,NaN
1,00001MG150289905,20151211.0,None,None,74949.0,None,NaN,6.0,SITE_DEFAULT_LOCAL,"HOLOGIC, Inc.",...,NaN,NaN,US,NaN,None,None,NaN,NaN,NaN,NaN
2,00001MG160004625,20160107.0,None,None,75551.0,None,NaN,48.0,SITE_DEFAULT_LOCAL,"HOLOGIC, Inc.",...,NaN,NaN,US,NaN,None,None,NaN,NaN,NaN,NaN
3,00001MG160004625,20160107.0,None,None,73527.0,None,NaN,6.0,SITE_DEFAULT_LOCAL,"HOLOGIC, Inc.",...,NaN,NaN,US,NaN,None,None,NaN,NaN,NaN,NaN
4,00001MG160004625,20160107.0,None,None,73202.0,None,NaN,2.0,SITE_DEFAULT_LOCAL,"HOLOGIC, Inc.",...,NaN,NaN,US,NaN,None,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9149,00006NM150024043,NaN,None,None,NaN,None,NaN,18.0,SITE_DEFAULT_LOCAL,ADAC,...,NaN,NaN,None,NaN,None,None,NaN,NaN,NaN,1292.0
9150,SJH3929401,20090915.0,None,None,202618.468,None,NaN,1001.0,SITE_DEFAULT_LOCAL,"FUJI PHOTO FILM Co., ltd.",...,NaN,NaN,OD,NaN,None,None,NaN,NaN,NaN,1292.0
9151,00006CT140009930,20140123.0,None,None,172557.768993,None,NaN,4.0,SITE_DEFAULT_LOCAL,SIEMENS,...,NaN,NaN,None,NaN,CW,None,25.0,3.000,NaN,1292.0
9152,00006CT200163452,20201102.0,None,None,101818.68,None,NaN,3.0,SITE_DEFAULT_LOCAL,SIEMENS,...,NaN,NaN,HU,NaN,CW,None,-674.7,5.000,NaN,NaN


In [17]:
row = combined_meta.iloc[100]
accession_number = row['AccessionNumber']
radiology_note = radiology_notes.loc[radiology_notes['EVENT_DOCUMENT_DESC'] == accession_number]
if len(radiology_note) == 0:
    print("No radiology note for accession number: ", accession_number)

#Append radiology note columns to combined_meta
for col in radiology_note.columns:
    combined_meta.at[idx, col] = radiology_note[col].values[0]

No radiology note for accession number:  00003DX170043130


IndexError: index 0 is out of bounds for axis 0 with size 0

In [21]:
acc = radiology_notes['ACC_NBR']
acc_comb = combined_meta['AccessionNumber']

In [22]:
common = set(acc).intersection(set(acc_comb))
common

{'00001US170030923',
 '00002DX120184445',
 '00001CT160233820',
 '00001CT200342234',
 '00001CT170269967',
 '00001DX200337908',
 '00001DX200213335',
 '00002DX150075442',
 '00001US190274975',
 '00006DX210210690',
 '00005DX170077185',
 '00001DX120102938',
 '00001DX110072713',
 '00002CT180157815',
 '00001CT200225640',
 '00001DX190329971',
 '00001CT190021243',
 '00001US140075226',
 '00001DX190015087',
 '00002DX120081625',
 '00001US110217138',
 '00006DX200008896',
 '00005CT160006654',
 '00006CT150103606',
 '00002DX110010222',
 '00002DX210133316',
 '00002DX110015482',
 '00001CT160236223',
 '00002DX150125516',
 '00006CT150114620',
 '00002DX190054155',
 '00002CT160108244',
 '00002DX140153904',
 '00001CT190149544',
 '00005CT180070685',
 '00001CT210063116',
 '00001CT140044937',
 '00002CT190106697',
 '00001CT170192819',
 '00001US150044324',
 '00006DX170020129',
 '00001DX170359018',
 '00001CT200041203',
 '00002CT180054500',
 '00002US110140963',
 '00001DX130211506',
 '00002CT210140651',
 '00002DX1700

In [15]:
radiology_notes.head()

,ACC_NBR,PATIENT_ID,EMPI_NBR,ENCNTR_ID,HNAM_DOCUMENT_CLINICAL_ID,HNAM_DOCUMENT_CLINICAL_NM,DAY_VERIFIED,DOC_TEXT,DOC_ABSTRACT,EVENT_DOCUMENT_DESC,EVENT_DOCUMENT_KEY
0,00006DX190104831,18007243,803350,156619493.0,2228789^EVENT,XR Chest 1 View Portable,06/28/2019,REPORT\r\rEXAM: XR Chest 1 View Portable\n\nES...,REPORT\r\rEXAM: XR Chest 1 View Portable\n\nES...,00006DX20190104831,127041997.0
1,00002DX200136398,27595956,5889242,174163224.0,2228795^EVENT,XR Chest 2 Views PA + Lat Stnd Protocol,07/23/2020,REPORT\r\rEXAM: XR Chest 2 Views PA + Lat Stnd...,REPORT\r\rEXAM: XR Chest 2 Views PA + Lat Stnd...,00002DX20200136398,140584055.0
2,00001CT190035194,18095426,891532,149573470.0,1460399309^EVENT,CT Neuro Spine Outside Image Ref Only,02/02/2019,CT NEURO SPINE OUTSIDE IMAGE REF ONLY\r\rRefer...,CT NEURO SPINE OUTSIDE IMAGE REF ONLY\r\rRefer...,00001CT20190035194,115489211.0
3,00002DX200041549,18302477,1098583,167980557.0,2228795^EVENT,XR Chest 2 Views PA + Lat Stnd Protocol,02/25/2020,ADDEND\r\rThis is the incorrect report as this...,ADDEND\r\rThis is the incorrect report as this...,00002DX20200041549,135818096.0
4,00002DX200138873,19065136,1861242,174284902.0,2228795^EVENT,XR Chest 2 Views PA + Lat Stnd Protocol,07/27/2020,REPORT\r\rEXAM: XR Chest 2 Views PA + Lat Stnd...,REPORT\r\rEXAM: XR Chest 2 Views PA + Lat Stnd...,00002DX20200138873,140714001.0


In [ ]:
for idx, row in combined_meta.iterrows():
    accession_number = row['AccessionNumber']
    radiology_note = radiology_notes.loc[radiology_notes['AccessionNumber'] == accession_number]
    if len(radiology_note) == 0:
        print("No radiology note for accession number: ", accession_number)
        continue

    #Append radiology note columns to combined_meta
    for col in radiology_note.columns:
        combined_meta.at[idx, col] = radiology_note[col].values[0]
    
path_to_combined_meta = Path('/labs/collab/Imaging/Imaging-PHI/Emory_Images/Meta/Combined_Meta_' + year + '_Radiology.pkl')
combined_meta.to_pickle(path_to_combined_meta)